In [0]:
from pyspark.sql.functions import *

In [0]:
# ingredient_agg = (
#     spark.table("retail_analytics.pizza.gold_ingredient_usage")
#     .groupBy("pizza_name_id")
#     .agg(
#         concat_ws(
#             ", ",
#             collect_list(
#                 concat(
#                     col("ingredient"),
#                     lit(" ("),
#                     round(col("total_grams_used"), 1),
#                     lit("g)")
#                 )
#             )
#         ).alias("ingredients_summary")
#     )
# )
# # display(ingredient_agg)

In [0]:
# from pyspark.sql.functions import col, concat, lit, round

# pizza_semantic_facts = (
#     spark.table("retail_analytics.pizza.gold_pizza_metrics")
#     .join(ingredient_agg, on="pizza_name_id", how="left")
#     .withColumn(
#         "document",
#         concat(
#             lit("The pizza "),
#             col("pizza_name_id"),
#             lit(" belongs to the "),
#             col("pizza_category"),
#             lit(" category. It sold "),
#             col("total_units_sold"),
#             lit(" units and generated revenue of "),
#             round(col("total_revenue"), 2),
#             lit(". The average price is "),
#             round(col("avg_price"), 2),
#             lit(". Ingredients include: "),
#             col("ingredients_summary"),
#             lit(".")
#         )
#     )
#     .select(
#         col("pizza_name_id"),
#         col("document")
#     )
# )
# # display(pizza_semantic_facts)

# pizza_semantic_facts.write \
#     .mode("overwrite") \
#     .format("delta") \
#     .option("mergeSchema", "true")\
#     .saveAsTable("retail_analytics.pizza.gold_semantic_pizza_facts")


In [0]:
from pyspark.sql.functions import col, concat, lit, round

ingredient_semantic_facts = (
    spark.table("retail_analytics.pizza.gold_ingredient_usage")
    .withColumn(
        "document",
        concat(
            lit("The ingredient "),
            col("pizza_ingredients"),
            lit(" is used in pizza "),
            col("pizza_name_id"),
            lit(". Total quantity used is "),
            round(col("total_ingredient_grams"), 1),
            lit(" grams. ")
        )
    )
    .select(
        col("pizza_name_id"),
        "document"
    )
)
# display(ingredient_semantic_facts)
ingredient_semantic_facts.write \
    .mode("overwrite") \
    .format("delta") \
    .option("mergeSchema", "true")\
    .saveAsTable("retail_analytics.pizza.gold_semantic_ingredient_facts")


In [0]:
ingredient_summary = (
    spark.table("retail_analytics.pizza.gold_ingredient_usage")
    .groupBy("pizza_ingredients")
    .agg(round(sum("total_ingredient_grams"), 1).alias("total_grams_used"))
    .withColumn(
        "document",
        concat(
            lit("The ingredient "),
            col("pizza_ingredients"),
            lit(" has been used a total of "),
            col("total_grams_used"),
            lit(" grams across all pizzas.")
        )
    )
    .select(col("pizza_ingredients"), col("document"))
)

ingredient_summary.write \
    .mode("overwrite") \
    .format("delta") \
    .option("mergeSchema", "true")\
    .saveAsTable("retail_analytics.pizza.gold_semantic_ingredient_summary")


In [0]:
# semantic_facts = (
#     spark.table("retail_analytics.pizza.gold_pizza_metrics")
#     .withColumn(
#         "document",
#         concat(
#             lit("The pizza "),
#             col("pizza_name_id"),
#             lit(" in category "),
#             col("pizza_category"),
#             lit(" sold "),
#             col("total_units_sold"),
#             lit(" units and generated revenue of "),
#             round(col("total_revenue"), 2),
#             lit(". The average price is "),
#             round(col("avg_price"), 2),
#             lit(".")
#         )
#     )
#     .select("pizza_name_id", "document")
# )

# semantic_facts.write \
#     .mode("overwrite") \
#     .format("delta") \
#     .saveAsTable("retail_analytics.pizza.gold_semantic_facts")
